# Energy Reconstruction Using CNN

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

## FIX THIS!
simPrefix = 'Users\kmays\simFiles'

## Data Input

In [2]:
x, y = load_preprocessed(simPrefix, 'train')

ZeroDivisionError: division by zero

In [3]:
print(x.shape)
print(y.keys())

(549773, 10, 10, 4)
dict_keys(['comp', 'energy', 'dir', 'plane_dir', 'laputop_dir', 'small_dir'])


In [4]:
# 85/15 split for training/testing
energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

## Model Training

### Alpha Model
- Input: no charge merge, no time layers included, normalized data
- Layers: Two convolutional
- Output: Energy

In [7]:
# Name for model
key = 'first_model'

# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':None, 't':False, 'normed':True}

In [8]:
# Create model
model = Sequential(name=nameModel(prep, 'test'))  # Automatic naming for flexible assessment later
# Add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(1)) # No activation function for last layer of regression model

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])

In [9]:
# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)
temp_y = energy

In [10]:
# Train
history = model.fit(x_i[trainCut], temp_y[trainCut], validation_data=(x_i[testCut], temp_y[testCut]), epochs=3)

Epoch 1/3
14611/14611 [==============================] - 67s 5ms/step - loss: 0.1588 - mae: 0.2530 - mse: 0.1588 - val_loss: 0.0661 - val_mae: 0.1999 - val_mse: 0.0661
Epoch 2/3
14611/14611 [==============================] - 64s 4ms/step - loss: 0.0652 - mae: 0.1978 - mse: 0.0652 - val_loss: 0.0644 - val_mae: 0.1948 - val_mse: 0.0644
Epoch 3/3
14611/14611 [==============================] - 62s 4ms/step - loss: 0.0556 - mae: 0.1821 - mse: 0.0556 - val_loss: 0.0517 - val_mae: 0.1770 - val_mse: 0.0517


In [11]:
# Save model to file for easy loading
## WHERE ARE YOU SAVING TO?
model.save('model_%s.h5' % key)

## Your task

- **Create your own model**
- Replace the model here w/ *simplified* form of Brandon's model (focus: including zenith)
- change the zenith input to cosine(zenith) input